In [1]:
%pip install pybamm
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pybamm
import pandas as pd
from pybamm import BaseModel
from IPython.display import display

## [1] Doyle Fuller Newman [2] Single Particle Model (SPM) [3] Single Particle Model with Electrolyte (SPMe)
availableModel = ["DFN", "SPM", "SPME"]
selectedModel = availableModel[0]
model: BaseModel

# Choose model
match selectedModel:
    case "DFN":
        model = pybamm.lithium_ion.DFN()
    case "SPM":
        model = pybamm.lithium_ion.SPM()
    case "SPME":
        model = pybamm.lithium_ion.SPMe()

# List of capacities from 5.0 Ah to 3.05 Ah in 1% steps
capacity_variants = [round(5.0 * (1 - i / 100), 2) for i in range(40)]

# Final DataFrame to store all results
all_results = pd.DataFrame()

# Loop through all capacity variants
for variant_num, cap in enumerate(capacity_variants, start=1):
    print(f"Running simulation for Variant {variant_num} with Q = {cap} Ah")

    selectedInputParameter = pybamm.ParameterValues("Chen2020")
    selectedInputParameter.update({
        "Nominal cell capacity [A.h]": cap
    })
    
    experiment = pybamm.Experiment(
        [
            (
                "Charge at C/2 until 4.2 V",
                "Discharge at C/6 until 2.5 V"
            )
        ],
        period="1 second"
    )

    sim = pybamm.Simulation(model, experiment=experiment, parameter_values=selectedInputParameter)
    
    try:
        sim.solve()
    except Exception as e:
        print(f"Simulation failed for Q = {cap} Ah: {e}")
        continue  # skip to the next variant

    # Prepare data collection
    output_variables = ["Time [s]", "Current [A]", "Voltage [V]", "X-averaged cell temperature [K]",
                        "BatteryCapacity", "TimeDiff", "CapacityDiff", "SOC"]
    
    data = {}
    num_rows = len(sim.solution["Time [min]"].entries)

    for var in output_variables:
        if var == "TimeDiff":
            timeDiffValues = [0] + [
                sim.solution["Time [s]"].entries[i] - sim.solution["Time [s]"].entries[i - 1]
                for i in range(1, num_rows)
            ]
            data[var] = timeDiffValues
        elif var == "BatteryCapacity":
            data[var] = [cap] * num_rows
        elif var == "CapacityDiff":
            capacityDiffValues = [0]
            for i in range(1, num_rows):
                deltaCapacity = sim.solution["Current [A]"].entries[i] * (timeDiffValues[i] / 3600) * -1
                capacityDiffValues.append(deltaCapacity)
            data[var] = capacityDiffValues
        elif var == "SOC":
            soc_values = [1]
            for i in range(1, num_rows):
                updated_soc = soc_values[i - 1] + (data["CapacityDiff"][i] / cap)
                soc_values.append(max(0, min(1, updated_soc)))
            data[var] = soc_values
        else:
            data[var] = sim.solution[var].entries

    df = pd.DataFrame(data)
    df["Variant"] = variant_num
    all_results = pd.concat([all_results, df], ignore_index=True)

# Save to CSV
all_results.to_csv("batch_battery_simulation.csv", index=False)
display(all_results)


Running simulation for Variant 1 with Q = 5.0 Ah
Running simulation for Variant 2 with Q = 4.95 Ah
Running simulation for Variant 3 with Q = 4.9 Ah
Running simulation for Variant 4 with Q = 4.85 Ah
Running simulation for Variant 5 with Q = 4.8 Ah
Running simulation for Variant 6 with Q = 4.75 Ah
Running simulation for Variant 7 with Q = 4.7 Ah
Running simulation for Variant 8 with Q = 4.65 Ah
Running simulation for Variant 9 with Q = 4.6 Ah
Running simulation for Variant 10 with Q = 4.55 Ah
Running simulation for Variant 11 with Q = 4.5 Ah
Running simulation for Variant 12 with Q = 4.45 Ah
Running simulation for Variant 13 with Q = 4.4 Ah
Running simulation for Variant 14 with Q = 4.35 Ah
Running simulation for Variant 15 with Q = 4.3 Ah
Running simulation for Variant 16 with Q = 4.25 Ah
Running simulation for Variant 17 with Q = 4.2 Ah
Running simulation for Variant 18 with Q = 4.15 Ah
Running simulation for Variant 19 with Q = 4.1 Ah
Running simulation for Variant 20 with Q = 4.05 Ah

,Time [s],Current [A],Voltage [V],X-averaged cell temperature [K],BatteryCapacity,TimeDiff,CapacityDiff,SOC,Variant
0,0.000000,0.833333,4.144409,298.15,5.00,0.000000,0.000000,1.000000,1
1,1.000000,0.833333,4.143481,298.15,5.00,1.000000,-0.000231,0.999954,1
2,2.000000,0.833333,4.142612,298.15,5.00,1.000000,-0.000231,0.999907,1
3,3.000000,0.833333,4.141797,298.15,5.00,1.000000,-0.000231,0.999861,1
4,4.000000,0.833333,4.141031,298.15,5.00,1.000000,-0.000231,0.999815,1
...,...,...,...,...,...,...,...,...,...
1112674,35974.000000,0.508333,2.502233,298.15,3.05,1.000000,-0.000141,0.000000,40
1112675,35975.000000,0.508333,2.501581,298.15,3.05,1.000000,-0.000141,0.000000,40
1112676,35976.000000,0.508333,2.500929,298.15,3.05,1.000000,-0.000141,0.000000,40
1112677,35977.000000,0.508333,2.500276,298.15,3.05,1.000000,-0.000141,0.000000,40
